In [ ]:
from openpipe import OpenAI
from dotenv import load_dotenv
import os
from datetime import datetime, timezone
from pathlib import Path
from memo import memlist, memfile, grid, time_taken
import json

load_dotenv()

client = OpenAI(
  openpipe={"api_key": os.environ["OPENPIPE_API_KEY"]}
)

results_fpath = Path("../data/results.jsonl")

if results_fpath.exists():
    data = [json.loads(l) for l in results_fpath.read_text().splitlines()]
else:
    data = []
    
system_prompt = """You are an expert poet.\nYou write 4 line poems in ABBA schema.\nThe A lines must not rhyme with the B lines.\nYou return only the poem.\nDo not return (A) / (B) indicators."""

structured_system_prompt = """
You are an expert poet.
You write 4 line poems in ABBA schema.
The A lines must not rhyme with the B lines.
Return your result in json format according to the schema below.
Return only the json, no extra text, no codefences.

from pydantic import BaseModel, Field
class ABBAPoem(BaseModel):
    rhyme_word_A1: str = Field(description="The word that sentence A1 ends with.")
    sentence_A1: str = Field(
        description="The sentence that ends with the rhyme word A1."
    )
    rhyme_word_B1: str = Field(
        description="The word that sentence B1 ends with. It SHOULD NOT rhyme with rhyme_word_A1."
    )
    sentence_B1: str = Field(
        description="The sentence that ends with the rhyme word B1."
    )
    rhyme_word_B2: str = Field(
        description="The word that sentence B2 ends with. It SHOULD rhyme with rhyme_word_B1. It SHOULD NOT rhyme with rhyme_word_A1."
    )
    sentence_B2: str = Field(
        description="The sentence that ends with the rhyme word B2."
    )
    rhyme_word_A2: str = Field(
        description="The word that sentence A2 ends with. It SHOULD rhyme with rhyme_word_A1. It SHOULD NOT rhyme with rhyme_word_B2."
    )
    sentence_A2: str = Field(
        description="The sentence that ends with the rhyme word A2."
    )
"""


@memfile(filepath=results_fpath)
@memlist(data=data)
@time_taken()
def create_poem(model, topic):
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": structured_system_prompt
            },
            {
                "role": "user",
                "content": topic
            }
        ],
        temperature=0,
        openpipe={
            "tags": {
                "prompt_id": "counting",
                "any_key": "any_value"
            }
        },
    )

    output = completion.choices[0].message.content
    output = json.loads(output)
    poem = "\n".join((output['sentence_A1'], output['sentence_B1'], output['sentence_B2'], output['sentence_A2']))
    return {"poem": poem, "datetime": datetime.now(timezone.utc).isoformat(), "format": "fine-tuned"}

In [ ]:
POEM_TOPICS = (
    "dogs",
    "cats",
    "spring",
    "summer",
    "christmas",
    "halloween",
    "love",
    "loss",
    "family",
    "friends",
    "war",
    "peace",
    "nature",
    "city",
    "home",
    "work",
    "school",
    "music",
    "art",
    "food",
    "winter",
    "fall",
    "dreams",
    "life",
    "death",
    "childhood",
    "time",
    "memories",
    "happiness",
    "sadness",
    "adventures",
    "travels",
    "fantasy",
    "realities",
    "freedom",
    "night",
    "day",
    "ocean",
    "mountains",
    "stars",
    "moon",
    "birth",
    "ageing",
    "loneliness",
    "hope",
)

In [ ]:
for topic in POEM_TOPICS:
    create_poem(model="openpipe:mistral-7b-structured-output-123", topic=topic)